# Parsing TEDDY's Sequence Read Archive (SRA) data from dbGaP

We requested the "microbiome data" from dbGaP listed here: https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=phs001442.v4.p3.
Once the access was granted, we used the dbGaP authorized access to request the microbiome data.
We received a big list of 92096 of SRA accession ids, each one refering to a sequencing run. We started downloading the data in small batches.

### The problem

- After downloading 10 batches of ~200 fastq files, I started processing the files, and I noticed that the data that we downloaded wasn't microbiome 16S amplicon, nor metagenome. It was blood transcriptomics!!! 
- I discovered that because fastqc pointed out weird issues, then I use pysradb to extract detailed metadata from a few randomly selected accession ids from our big list. Some of them were samples from blood, some of them were from stool, plasma, nasal, etc. So we needed a more systematic and automated way of spliting and organizing this mess!

### The solution

- By using pysradb I downloaded very detailed metadata on TEDDY's BioProject ids. So I loaded the SRA accession id metadata that I got from dbGaP authorized access via run selector (https://0-www-ncbi-nlm-nih-gov.brum.beds.ac.uk/Traces/study/) and I found out what are the BioProject ids assigned to TEDDY: ['PRJNA400115', 'PRJNA416160']. There are thousands of run ids (sample ids) associated to each project id, and pysradb downloads the detailed metadata for each one of them as well, which is nice.
- After that, I used pysradb to retrieve very detailed metadata on all samples of both BioProject ids, especially:
    - histological type: where the sample was collected from (stool, blood, plasma, nasal swab, etc.)
    - library_source: type of sequencing library source (metagenomic, transcriptomic, genomic, etc.)
    - library_strategy: type os sequencing library strategy (WGS, amplicon, other). 
    - experiment_title: the type of the experiment (gut metagenomics, gut 16S, gut virome, transcriptome, etc.)

In [1]:
import pandas as pd

In [2]:
#this txt file was aquired from the SRA run selector tool via dbGap authorized access
teddy_metadata_path = "/blue/raquel.dias/data/TEDDY/dbGaP/download/SraRunTable_fastq.txt"
teddy_sra_metadata = pd.read_csv(teddy_metadata_path, low_memory=False)
#teddy_sra_metadata.columns
teddy_sra_metadata['BioProject'].unique()

array(['PRJNA400115', 'PRJNA416160'], dtype=object)

In [3]:
#this will take almost one hour, skip this step and use the already downloaded file if you have it
#!pysradb metadata --detailed PRJNA400115  --saveto PRJNA400115_teddy_pysradb_metadata.tsv
#!pysradb metadata --detailed PRJNA416160  --saveto PRJNA416160_teddy_pysradb_metadata.tsv

In [4]:
PRJNA400115_metadata = pd.read_csv('PRJNA400115_teddy_pysradb_metadata.tsv', sep='\t')
PRJNA416160_metadata = pd.read_csv('PRJNA416160_teddy_pysradb_metadata.tsv', sep='\t')
PRJNA400115_metadata.columns

Index(['run_accession', 'study_accession', 'study_title',
       'experiment_accession', 'experiment_title', 'experiment_desc',
       'organism_taxid', 'organism_name', 'library_name', 'library_strategy',
       'library_source', 'library_selection', 'library_layout',
       'sample_accession', 'sample_title', 'instrument', 'instrument_model',
       'instrument_model_desc', 'total_spots', 'total_size', 'run_total_spots',
       'run_total_bases', 'run_alias', 'experiment_alias', 'gap_accession',
       'submitter handle', 'biospecimen repository', 'study name',
       'study design', 'biospecimen repository sample id',
       'submitted sample id', 'submitted subject id', 'gap_sample_id',
       'gap_subject_id', 'sex', 'body site', 'histological type',
       'analyte type', 'gap_consent_code', 'gap_consent_short_name',
       'ena_fastq_http', 'ena_fastq_http_1', 'ena_fastq_http_2',
       'ena_fastq_ftp', 'ena_fastq_ftp_1', 'ena_fastq_ftp_2'],
      dtype='object')

In [5]:
PRJNA416160_metadata['experiment_title'].unique()

array(['Human gut primary virome sequencing: Sample M1000518',
       'Human gut primary virome sequencing: Sample M1001368',
       'Human gut primary virome sequencing: Sample M1003303', ...,
       'Human WGS Aligned Sequence Data: Sample WGS_99642150_P1',
       'Human WGS Aligned Sequence Data: Sample WGS_73666079_P1',
       'Human WGS Aligned Sequence Data: Sample WGS_82388918_P1'],
      dtype=object)

In [6]:
#extract preffix of experiment title
PRJNA400115_metadata['experiment_title'] = PRJNA400115_metadata['experiment_title'].str.replace(':.*','', regex=True)
PRJNA416160_metadata['experiment_title'] = PRJNA416160_metadata['experiment_title'].str.replace(':.*','', regex=True)

In [7]:
PRJNA416160_metadata['experiment_title'].unique()

array(['Human gut primary virome sequencing',
       'Human plasma primary virome sequencing',
       'Human nasal primary virome sequencing',
       'Human gut cultured virome sequencing',
       'Human plasma cultured virome sequencing',
       'Human nasal cultured virome sequencing',
       'Human stool ITS2 sequencing', 'Human nasal ITS2 sequencing',
       'Human nasal metagenomic WGS', 'Human stool deep metagenomic wgs',
       'Human nasal 16S rRNA', 'Human Aligned RNA-Seq Data',
       'Human WGS Aligned Sequence Data'], dtype=object)

In [8]:
#I used this function to dig trhough the data and understant its structure
def show_unique_values(metadata):
    unique_val_list = []
    for column_name in metadata.columns:
        unique_values = metadata[column_name].unique()
        unique_val_count = len(unique_values)
        if unique_val_count<10 and unique_val_count>1:
            if column_name.startswith('ena_')==False and column_name!='sex':
                print(column_name, list(unique_values))
                print(metadata[column_name].value_counts())
                print('\n')

In [9]:
#show_unique_values(PRJNA416160_metadata)
#show_unique_values(PRJNA400115_metadata)

In [19]:
#Once I understood the data, I used this function to split the TEDDY accession ids
def split_sra_bioproject_by_experiment(pysradb_metadata_df):
    total_run_count = 0
    for histological_type in pysradb_metadata_df['histological type'].unique():
        histological_type_metadata = pysradb_metadata_df.loc[pysradb_metadata_df['histological type']==histological_type]
        for library_source in histological_type_metadata['library_source'].unique():
            library_source_metadata = histological_type_metadata.loc[histological_type_metadata['library_source']==library_source]
            for library_strategy in library_source_metadata['library_strategy'].unique():
                library_strategy_metadata = library_source_metadata.loc[library_source_metadata['library_strategy']==library_strategy]
                data_type_list = [histological_type, library_source, library_strategy]
                data_type_string = '__'.join(data_type_list).replace(' ','_')
                print(data_type_string)
                count_per_experiment = dict(library_strategy_metadata['experiment_title'].value_counts())
                for key, value in count_per_experiment.items():
                    total_run_count += value
                    accession_list_name = data_type_string+'__'+key.replace(' ','_')
                    #print(accession_list_name)
                    experiment_metadata = library_strategy_metadata.loc[library_strategy_metadata['experiment_title']==key]
                    experiment_metadata_accession_list = list(experiment_metadata['run_accession'])
                    print("      ",key.replace(' ','_')+"\t", value)
                    outfile = open(accession_list_name+'.txt','w')
                    outfile.write('\n'.join(experiment_metadata_accession_list)+'\n')
                    outfile.close()
                print("")
    print("total_run_count\t", total_run_count)

In [20]:
split_sra_bioproject_by_experiment(PRJNA400115_metadata)

Stool__METAGENOMIC__WGS
       Human_gut_metagenomics	 13245

Stool__GENOMIC__OTHER
       Human_gut_16S_rRNA	 13714

total_run_count	 26959


In [21]:
split_sra_bioproject_by_experiment(PRJNA416160_metadata)

Stool__METAGENOMIC__WGS
       Human_gut_cultured_virome_sequencing	 12862
       Human_gut_primary_virome_sequencing	 12685
       Human_stool_deep_metagenomic_wgs	 1194

Stool__METAGENOMIC__AMPLICON
       Human_stool_ITS2_sequencing	 12453

Plasma__METAGENOMIC__WGS
       Human_plasma_cultured_virome_sequencing	 6220
       Human_plasma_primary_virome_sequencing	 5907

Nasal_Swab__METAGENOMIC__WGS
       Human_nasal_cultured_virome_sequencing	 1728
       Human_nasal_primary_virome_sequencing	 1725
       Human_nasal_metagenomic_WGS	 1006

Nasal_Swab__METAGENOMIC__AMPLICON
       Human_nasal_16S_rRNA	 1728
       Human_nasal_ITS2_sequencing	 1470

Blood__TRANSCRIPTOMIC__RNA-Seq
       Human_Aligned_RNA-Seq_Data	 5034

Blood__GENOMIC__WGS
       Human_WGS_Aligned_Sequence_Data	 1125

total_run_count	 65137


In [13]:
# 26959 + 65137 = 92096
# Matches to the number found here: https://www.ncbi.nlm.nih.gov/sra/?term=teddy